In [1]:
from tinysmpc import VirtualMachine, PrivateScalar

# Private 2 Party State Machine

This notebook demonstrates how a state machine can be privately evaluated using two non-colluding servers. For this protocol "privacy" is defined as:
- The **servers** do not learn the **input** of the client
- The **servers and client** do not know the **current state** of the machine (apart from the initial state)

## General Protocol Overview

### Preprocessing: Convert Regex to Arithmetic Circuit
1. Regex is converted into a [DFA](https://en.wikipedia.org/wiki/Deterministic_finite_automaton) (state machine)
2. DFA is converted into a polynomial equation over a finite field (or ring)
3. Polynomial converted into arithmetic circuit
4. Arithmetic circuit is given each server

![Preprocessing](./images/preprocessing.png)
*Arithmetic Circuit image By User:Derfel73; User:Tcshasaposse - User:Tcshasaposse, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=25138015*

### Evaluation: Client Inputs Token
1. Client inputs a token (e.g. a character)
2. The token is additively secret shared with servers ($token_0$ shared with $server_0$ and $token_1$ shared with $server_1$, where $token_0 + token_1 = input$)
3. Servers evaluate the arithmetic circuit using their own individual input share as well as their share of the current state
4. If more input go to step 1 
5. If evaluation is complete combine the results from the servers to get the final state

![Evaluation Steps](./images/evaluation_steps.png)
*Arithmetic Circuit image By User:Derfel73; User:Tcshasaposse - User:Tcshasaposse, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=25138015*

## **STEP 1:** Regex to State Machine

For this demo we will be basing our state machine off the regex expression "ab". This machine will only accept the string "ab" and nothing else. The state machine can be visualized as follows:

![transition graph of state machine for regex "ab"](./images/ab_transition_graph.png)
<!-- Generated using https://ivanzuzak.info/noam/webapps/fsm_simulator/ -->

### State Table for "ab"

|             | **INPUT 'a'** | **INPUT 'b'** |
| ----------- | ------------- | ------------- |
| **STATE 0** | STATE 2       | STATE 3       |
| **STATE 1** | STATE 3       | STATE 3       |
| **STATE 2** | STATE 3       | STATE 1       |
| **STATE 3** | STATE 3       | STATE 3       |

## **STEP 2:** State Machine to Polynomial Representation

To interpolate we can visualize the state transition table as a bunch of 3D points.

| **STATE** | **INPUT ('a'=0 and 'b'=1)** | **NEXT STATE** |
| --------- | --------------------------- | -------------- |
| 0         | 0                           | 2              |
| 1         | 0                           | 3              |
| 2         | 0                           | 3              |
| 3         | 0                           | 3              |
| 0         | 1                           | 3              |
| 1         | 1                           | 3              |
| 2         | 1                           | 1              |
| 3         | 1                           | 3              |

We can now use bilinear interpolation to construct this equation:

$$f(x, y) = 2 + 1y + 10922x + 54602xy + 65520x^2 + 65518x^2y + 54601x^3 + 10921x^3y$$

This equation may look complicated but it is simply the state transition table represented as a polynomial function over a Galois ring where $p = 65521$. 

In [2]:
PRIME = 65521

def tf(s: int, t: int):
    """
    Transition function for the regex example 'ab' over a Galois ring where p = 65521.
    
    (STATE, INPUT) -> NEXT STATE

    Args:
        s : The current state
        t : The input token
    """

    return 2 + 1*t + 10922*s + 54602*s*t + 65520*(s**2) + 65518*(s**2)*t + 54601*(s**3) + 10921*(s**3)*t

def token_to_int(token: str):
    """Converts a token to an integer so that it can be read by the state machine
    
    Args:
        token : The token to convert
    """
    if token == 'a':
        return 0
    elif token == 'b':
        return 1
    else:
        raise ValueError('Invalid token')


def eval_state_machine(token, shared_state, tf, PRIME, alice, bob, charlie):
    """Evaluates the state machine using the given transition function
    
    Args:
        token(str) : The token to evaluate
        shared_state : The current shared state of the state machine
        tf : The transition function
        PRIME : The prime used for the Galois ring
        alice : Server 0
        bob : Server 1
        charlie : The client sending the token
    """
    token_int = token_to_int(token)
    token = PrivateScalar(token_int, charlie)       # client inputs new token
    shared_token = token.share([alice, bob], PRIME) # client shares token with servers
    return tf(shared_state, shared_token)           # servers compute new state using shares of current state and token

In [3]:
alice = VirtualMachine('alice')     # server 0
bob = VirtualMachine('bob')         # server 1
charlie = VirtualMachine('charlie') # client

shared_state = PrivateScalar(0, alice).share([alice, bob], PRIME) # initial state

# Evaluate the state machine on the input 'aaa'
shared_state = eval_state_machine('a', shared_state, tf, PRIME, alice, bob, charlie)
shared_state = eval_state_machine('a', shared_state, tf, PRIME, alice, bob, charlie)
shared_state = eval_state_machine('a', shared_state, tf, PRIME, alice, bob, charlie)

print(f"Reconstructed state from shares: {shared_state.reconstruct(alice)}") # reconstruct state from shares

values [37358]
values [41025]
self.value - mul 41025
other_value - mul 1
self.value - mul 24496
other_value - mul 1
self.value 41025
other_value 2
self.value - mul 37358
other_value - mul 10922
self.value - mul 28163
other_value - mul 10922
self.value 24496
other_value 40712
self.value 41027
other_value 24809
self.value - mul 37358
other_value - mul 54602
self.value - mul 28163
other_value - mul 54602
25485
values [40502]
values [51589]
values [35580]
self.value - mul 40502
other_value - mul -1
self.value - mul 34794
other_value - mul -1
self.value 43777
other_value 25019
self.value 21744
other_value 30727
self.value - mul 51589
other_value - mul -1
self.value - mul 45874
other_value - mul -1
self.value 24496
other_value 13932
self.value 41025
other_value 19647
self.value 3275
other_value 0
self.value 3275
other_value 52471
self.value 38428
other_value 0
self.value 38428
other_value 60672
self.value - mul 51589
other_value - mul 55746
self.value - mul 45874
other_value - mul 55746
self

ValueError: GF(65521) scalars must be in `0 <= x < 65521`, not 1871894934.

In [ ]:
state = 0 # initial state

state = tf(state, token_to_int('a')) % PRIME
state = tf(state, token_to_int('a')) % PRIME
state = tf(state, token_to_int('a')) % PRIME

print(f"State from direct evaluation: {state}")
print("State reconstructed from shares matches state from direct evaluation:", shared_state.reconstruct(alice).value == state)

State from direct evaluation: 3
<class 'int'>
<class 'int'>
State reconstructed from shares matches state from direct evaluation: True


In [ ]:
import galois
xx = galois.GF(7)
xx(3) * -1

GF(4, order=7)